The Blocks Puzzle
===============

The _Blocks Puzzle_ is a simplified version of _Tetris_ with a simple set of rules:

* The game is played on a rectangular board with fixed width and height.
* A sequence of rectangular blocks must be placed on the board. The blocks cannot be rotated.
* Every block can be placed anywhere on the board as long as the space is not occupied.
* After placing a block, all completed rows and columns are cleared.

In this project, you will have to implement an [online algorithm](https://en.wikipedia.org/wiki/Online_algorithm) for the Block Puzzle. The algorithm will read a sequence of blocks, i.e. ($w$,$h$)-pairs representing rectangles, and will place them on the board. Since the algorithm must be _online_, the location for each block must be computed without knowing the following blocks in the sequence.

You must create the class `MyPlayer` with an API compatible with the methods used in this notebook.

In [1]:
from myplayer import *

Let us create a 6x5 board (width=6, height=5). The square (0,0) is the one at the lower-left corner.

In [2]:
p = MyPlayer(6, 5)
print(p)

We can now place a 3x2-block in location (2,0)

In [3]:
p.place_block(Location(2, 0), Shape(3, 2))
print(p)

And few more blocks:

In [4]:
p.place_block(Location(1,5),Shape(1,4)).place_block(Location(4,4),Shape(1,1)).place_block(Location(0,2),Shape(3,2))
print(p)

If you want to put a block in an illegal position, an exception will be raised.

In [5]:
try:
    p.place_block(Location(0,1), Shape(2,2))
except:
    print("The block could not be placed. The board has not been modified")
print(p)

Now we will put a 2x2-block in location (2,3). The completed rows and columns will be cleared.

In [6]:
p.place_block(Location(2,3),Shape(2,2))
print(p)

## Let the computer play

Now you know the rules of this game. Let us allow the computer to play the game. For that, an algorithm must be designed to solve the following problem:

> Given a board and a new block, find a location to place the block.

Several strategies can be devised to find the best location. In this project we ask the algorithms to be deterministic. In short, a [deterministic algorithm](https://en.wikipedia.org/wiki/Deterministic_algorithm) may not take random choices, i.e., for the same board configuration and the same block it must take the same decision.

We ask every student to implement at least one algorithm, called _"simple"_, with the following specification:

> From all the possible locations for a block, pick the one with the lowest row. In case several locations are possible in the same row, pick the one with the lowest column.

Let us play the game in a small 4x4-board.

In [7]:
p = MyPlayer(4,4)

We let the computer choose the best location for a 2x2-Block. For that, we will used the method `play`that receives a block, i.e., a shape ($w$,$h$). The method returns the selected location, or `None` if the block cannot be placed anywhere.

In [8]:
block = Shape(2,2)
location = p.play(block)
if location is not None:
    print("Selected location:", location)
    p.place_block(location, block)
else:
    print("The block could not be placed")
print(p)

And now we play with more blocks from a list until one of them cannot be placed.

**Recommendation:** for debugging purposes, try to play the game with the same pieces mimicking the _simple_ strategy to find the locations. Check that the final result is the same.

In [9]:
blocks = [Shape(1,1), Shape(2,3), Shape(1,2), Shape(3,1), Shape(2,2), Shape(1,3),\
          Shape(1,3), Shape(3,2), Shape(1,1), Shape(2,1), Shape(1,3)]
for bl in blocks:
    location = p.play(bl)
    if location is None:
        print(bl, "could not be placed.")
        break
    print("Block placed in:", location)
    p.place_block(location, bl)
print(p)

We now define a function that plays the game for a list of blocks using a player.

In [10]:
def play_game(player, blocks, show=True):
    """It plays the blocks puzzle using a pre-defined player. If show is asserted, the state of the player 
    is printed after placing each block. It returns the number of blocks that could be placed.
    """
    if show: print(player)
    count = 0
    for block in blocks:
        assert player.is_legal(block)
        loc = player.play(block)
        if loc is None: break
        player.place_block(loc, block)
        count += 1
        if show: print(player)
    return count

## Reading a sequence of blocks from a file

Playing with long sequences of blocks could be a tedious task. The next function reads a file from an URL and converts it into a list of shapes. The file contains pairs of integer values (width and height), for example:
```
1 1  2 3  1 2
3 1  2 2  1 3  1 3
3 2  1 1  2 1  1 3
```

In [11]:
import urllib.request

def read_file(file, url=True):
    """It reads a file of integers representing shapes. Each pair of consecutive integers represents
    a shape (width and height). It returns a list of shapes. If url is not asserted, file is assumed
    to be the name of a local file.
    """
    if url:
        with urllib.request.urlopen(file) as reader:
            items = reader.read().split()
    else:
        with open(file) as reader:
            items = reader.read().split()
            
    # Check there is an even number of items
    assert len(items) % 2 == 0, "Wrong number of items in " + file
    
    # Convert pairs of items to shapes, checking that the items are correct.
    R = []
    for i in range(0, len(items), 2):
        # Check the items are numbers
        assert items[i].isdigit() and items[i+1].isdigit(), "Some element in the list is not an integer"
        w, h = int(items[i]), int(items[i+1])
        assert w > 0 and h > 0, "Illegal size for a shape"
        R.append(Shape(w, h))
    return R

You can download four sequences of blocks here: [blocks1.in](https://www.cs.upc.edu/~jordicf/Teaching/AP2/blocks1.in), [blocks2.in](https://www.cs.upc.edu/~jordicf/Teaching/AP2/blocks2.in), [blocks3.in](https://www.cs.upc.edu/~jordicf/Teaching/AP2/blocks3.in) and [blocks4.in](https://www.cs.upc.edu/~jordicf/Teaching/AP2/blocks4.in).

Each of these files contains a sequence of 1000 blocks. We can now play with them and see how many pieces we can place on a 20x20-board for each file.

In [12]:
url = "https://www.cs.upc.edu/~jordicf/Teaching/AP2/"
files = ["blocks1.in", "blocks2.in", "blocks3.in", "blocks4.in"]
for file in files:
    blocks = read_file(url+file, url=True)
    player = MyPlayer(20, 20)
    nblocks = play_game(player, blocks, False)
    print(nblocks, " blocks placed (out of ", len(blocks), ") for ", file, sep='')
    if nblocks < len(blocks):
        print("The block", blocks[nblocks], "cannot be placed.")
    print(player, '\n')

235 blocks placed (out of 1000) for blocks1.in
The block Shape(width=3, height=3) cannot be placed.
⬛⬛⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬛⬛⬜⬜⬜⬜⬛⬛⬛⬜⬜⬜⬜⬜⬛⬛⬛⬜⬜
⬛⬛⬛⬜⬜⬛⬛⬛⬛⬛⬜⬜⬛⬛⬛⬛⬛⬛⬜⬜
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜
⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬜⬜⬛⬛⬛⬛⬛⬛⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜
⬜⬜⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜
⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜ 

361 blocks placed (out of 1000) for blocks2.in
The block Shape(width=3, height=3) cannot be placed.
⬛⬛⬛⬛⬜⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬛⬛⬛⬜⬜⬜⬜⬛⬛⬛⬛⬛⬜⬜⬜⬛⬛⬛⬛
⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬜⬜⬛⬛⬛
⬛⬛⬜⬜⬜⬜⬜⬜⬛⬛⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜
⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜
⬛⬛⬜⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬜⬜⬜⬜⬜⬜⬛⬛⬜⬜⬛⬛
⬜⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬜
⬜⬜⬜⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛


## Blocks Pro (for free)

As expected, the _simple_ algorithm is not very smart. As an optional task (to be eligible for the maximum grade), we will allow each student to devise an _expert_ algorithm that can perform a better job than the simple one. This means that the expert algorithm should at least double the number of pieces placed by the simple algorithm.

For that, we will allow `MyPlayer` to accept a third parameter (a string) that will select the new algorithm. If the parameter is not defined, the `"simple"` value will be assumed by default. The `"expert"` value will select the new algorithm. This algorithm will only affect the selection of the location calculated by the `play` method.

Here you can see the results obtained by Jordi Cortadella's expert algorithm on various square boards. The algorithm manages to place all pieces of the four sequences in a 15x15 board. Can you beat it?

**Constraint:** The smart strategy used by the expert algorithm should be fast. In this project, the constraint will be determined by the runtime of the last cell of this notebook: it must be executed in less than 3 minutes (see the _Wall time_ at the output). As a clue, the runtime of the last cell using Jordi's algorithm is about 45 secs. 

In [13]:
%%time
methods = ["simple", "expert"]
width = range(10,16)

for w in width:
    for f in files:
        print(w, 'x', w, ' ', f, ': ', sep='', end='')
        blocks = read_file(url+f, url=True)
        for m in methods:
            player = MyPlayer(w, w, method=m)
            nblocks = play_game(player, blocks, False)
            print(m,' (', nblocks, ' blocks) ', sep='', end='')
        print()
    print("-"*60)

10x10 blocks1.in: simple (47 blocks) expert (160 blocks) 
10x10 blocks2.in: simple (38 blocks) expert (419 blocks) 
10x10 blocks3.in: simple (43 blocks) expert (629 blocks) 
10x10 blocks4.in: simple (73 blocks) expert (519 blocks) 
------------------------------------------------------------
11x11 blocks1.in: simple (48 blocks) expert (136 blocks) 
11x11 blocks2.in: simple (57 blocks) expert (1000 blocks) 
11x11 blocks3.in: simple (48 blocks) expert (165 blocks) 
11x11 blocks4.in: simple (46 blocks) expert (588 blocks) 
------------------------------------------------------------
12x12 blocks1.in: simple (84 blocks) expert (1000 blocks) 
12x12 blocks2.in: simple (118 blocks) expert (1000 blocks) 
12x12 blocks3.in: simple (78 blocks) expert (959 blocks) 
12x12 blocks4.in: simple (108 blocks) expert (1000 blocks) 
------------------------------------------------------------
13x13 blocks1.in: simple (75 blocks) expert (997 blocks) 
13x13 blocks2.in: simple (150 blocks) expert (1000 blocks